## Importing Libraries

In [7]:
import pandas as pd
import numpy as np
import json
from pprint import pprint
import requests
from bs4 import BeautifulSoup
from functools import reduce
from pandas import json_normalize

## Loading the City_bike csv

In [8]:
city_bike = pd.read_csv('city_bike.csv')
city_bike

,empty_slots,free_bikes,id,latitude,longitude,bike_station,number_of_bikes
0,2,17,fb337bbed72e2be090071e199899b2be,43.665269,-79.319796,Queen St E / Woodward Ave,19
1,11,2,4ff88d5880e71aa40d34cfe5d09b0ca7,43.671420,-79.445947,Primrose Ave / Davenport Rd,13
2,2,19,a09c67c0b419654d907c9134b108e328,43.666224,-79.317693,Queen St E / Rhodes Ave,21
3,26,9,d6a9daee68070a8b106cfb598d81308c,43.653236,-79.376716,Bond St / Queen St E,35
4,9,15,8f8af40d9388c8a3962559e8681d3db7,43.663722,-79.380288,Church St / Alexander St,24
...,...,...,...,...,...,...,...
772,9,8,bf5d4d14e7619f5ff51e264e4095cc4b,43.650298,-79.366485,Princess St / Front St E,17
773,23,0,c53d12128be1f1022c4b78518cca7675,43.662453,-79.397838,St. George St / Willcocks St,23
774,23,2,d0f019574616cb2bce86924059a4c0f8,43.647043,-79.383141,York St / Wellington St W (2),25
775,4,10,479d21ebdac47eb8e30a209268e0bebe,43.644887,-79.456412,High Park Blvd / Parkside Dr,14


# Basic Response from API

In [9]:
with open ("/Users/user/Downloads/my_key", "r") as f:
    api_key = f.read()
    #print (api_key)

In [10]:
def send_request(lat, long, categories, radius):
    url = f"https://api.foursquare.com/v3/places/search?ll={lat},{long}&radius={radius}&query='dining'"

    headers = {
        "accept": "application/json",
        "Authorization": api_key}

    response = requests.get(url, headers=headers)
    data = response.json()
    #pprint(data)
    return data

In [11]:
"""I had to use only 100 records because the data to be extracted was to larger for my system to handle.
I had to literally interupt the kernel."""

city_bike2 = city_bike.head(100)

### For Loop to iterate through the various lat and long in the city_bike dataframe.

In [12]:
lst = []
for index, row in city_bike2.iterrows():
    #print(row['latitude'], row['longitude'])
    response = send_request(row['latitude'], row['longitude'],'dining',1000)
    lst.append(response)

### Creating a DataFrame 

In [13]:
fsq_df = pd.DataFrame()
for i in lst:
    df1 = pd.DataFrame(i['results'])
    fsq_df = pd.concat([fsq_df,df1])

In [14]:
fsq_df

,fsq_id,categories,chains,closed_bucket,distance,geocodes,link,location,name,related_places,timezone
0,4ae0c7a8f964a520638221e3,"[{'id': 13199, 'name': 'Indian Restaurant', 's...",[],VeryLikelyOpen,783,"{'main': {'latitude': 43.67205, 'longitude': -...",/v3/places/4ae0c7a8f964a520638221e3,"{'address': '1460 Gerrard St E', 'country': 'C...",Udupi Palace,{'parent': {'fsq_id': '4d1bcc3234125941cd53cb1...,America/Toronto
1,5859d15c1d21ba53f2f9b4ef,"[{'id': 13003, 'name': 'Bar', 'short_name': 'B...",[],VeryLikelyOpen,809,"{'drop_off': {'latitude': 43.67254, 'longitude...",/v3/places/5859d15c1d21ba53f2f9b4ef,"{'address': '1471 Gerrard St E', 'country': 'C...",Lake Inez,{},America/Toronto
2,4c1169396e5dc9b61b10b02d,"[{'id': 13031, 'name': 'Burger Joint', 'short_...",[],VeryLikelyOpen,377,"{'main': {'latitude': 43.666763, 'longitude': ...",/v3/places/4c1169396e5dc9b61b10b02d,"{'address': '1636 Queen St E', 'country': 'CA'...",The Burger's Priest,{},America/Toronto
3,4ad9ebdcf964a520e61b21e3,"[{'id': 13049, 'name': 'Diner', 'short_name': ...",[],VeryLikelyOpen,129,"{'main': {'latitude': 43.665121, 'longitude': ...",/v3/places/4ad9ebdcf964a520e61b21e3,"{'address': '1483 Queen St E', 'country': 'CA'...",Chick-N-Joy,{},America/Toronto
4,54302333498ed6ec0533ab40,"[{'id': 13140, 'name': 'Egyptian Restaurant', ...",[],VeryLikelyOpen,1001,"{'drop_off': {'latitude': 43.671737, 'longitud...",/v3/places/54302333498ed6ec0533ab40,"{'address': '226 Greenwood Ave', 'country': 'C...",Maha's Fine Egyptian Cuisine,{},America/Toronto
...,...,...,...,...,...,...,...,...,...,...,...
5,55610dcd498e1b69839a65d9,"[{'id': 13002, 'name': 'Bakery', 'short_name':...",[],VeryLikelyOpen,682,"{'drop_off': {'latitude': 43.661132, 'longitud...",/v3/places/55610dcd498e1b69839a65d9,"{'address': '3 Westmoreland Ave', 'country': '...",Field Trip Cafe,{},America/Toronto
6,56418c69498ec00425302f6a,"[{'id': 13263, 'name': 'Japanese Restaurant', ...",[],LikelyOpen,876,"{'main': {'latitude': 43.649644, 'longitude': ...",/v3/places/56418c69498ec00425302f6a,"{'address': '1330 Dundas St W', 'country': 'CA...",Imanishi Japanese Kitchen,{},America/Toronto
7,5963b3e52be425495a1ed8a1,"[{'id': 13236, 'name': 'Italian Restaurant', '...",[],VeryLikelyOpen,777,"{'drop_off': {'latitude': 43.658444, 'longitud...",/v3/places/5963b3e52be425495a1ed8a1,"{'address': '1281 Bloor St W', 'country': 'CA'...",Sugo,{},America/Toronto
8,4dc5bc33d164eb9c9ff7a2a3,"[{'id': 13002, 'name': 'Bakery', 'short_name':...",[],VeryLikelyOpen,828,"{'drop_off': {'latitude': 43.661695, 'longitud...",/v3/places/4dc5bc33d164eb9c9ff7a2a3,"{'address': '205 Delaware Ave', 'country': 'CA...",Bakerbots Baking,{},America/Toronto


## Data Cleaning Process

Some of the columns in the Dataframe had a list of nested dictionary,to extract the data need i used the json_normalize( ) function.

In [15]:
fsq_df_normalized = json_normalize(fsq_df['geocodes'])
fsq_df_normalized

,main.latitude,main.longitude,drop_off.latitude,drop_off.longitude,roof.latitude,roof.longitude
0,43.672050,-79.322451,NaN,NaN,NaN,NaN
1,43.672346,-79.320804,43.672540,-79.320847,43.672346,-79.320804
2,43.666763,-79.315564,NaN,NaN,43.666709,-79.315659
3,43.665121,-79.321347,NaN,NaN,43.665121,-79.321347
4,43.671682,-79.328462,43.671737,-79.328337,43.671682,-79.328462
...,...,...,...,...,...,...
976,43.661214,-79.430286,43.661132,-79.430492,43.661214,-79.430286
977,43.649644,-79.426516,NaN,NaN,43.649644,-79.426516
978,43.658319,-79.442260,43.658444,-79.442286,43.658319,-79.442260
979,43.661781,-79.427899,43.661695,-79.428101,43.661781,-79.427899


In [16]:
# Dropping unwanted columns
fsq_df_normalized.drop(columns = ['drop_off.latitude','drop_off.longitude','roof.latitude','roof.longitude'], axis = 1, inplace = True)

In [17]:
# Renaming the columns

fsq_df_normalized.rename(columns = {'main.latitude':'latitude','main.longitude':'longitude'}, inplace = True)
fsq_df_normalized

,latitude,longitude
0,43.672050,-79.322451
1,43.672346,-79.320804
2,43.666763,-79.315564
3,43.665121,-79.321347
4,43.671682,-79.328462
...,...,...
976,43.661214,-79.430286
977,43.649644,-79.426516
978,43.658319,-79.442260
979,43.661781,-79.427899


In [18]:
fsq_df['latitude'] = fsq_df_normalized['latitude']
fsq_df['longitude'] = fsq_df_normalized['longitude']
fsq_df

,fsq_id,categories,chains,closed_bucket,distance,geocodes,link,location,name,related_places,timezone,latitude,longitude
0,4ae0c7a8f964a520638221e3,"[{'id': 13199, 'name': 'Indian Restaurant', 's...",[],VeryLikelyOpen,783,"{'main': {'latitude': 43.67205, 'longitude': -...",/v3/places/4ae0c7a8f964a520638221e3,"{'address': '1460 Gerrard St E', 'country': 'C...",Udupi Palace,{'parent': {'fsq_id': '4d1bcc3234125941cd53cb1...,America/Toronto,43.672050,-79.322451
1,5859d15c1d21ba53f2f9b4ef,"[{'id': 13003, 'name': 'Bar', 'short_name': 'B...",[],VeryLikelyOpen,809,"{'drop_off': {'latitude': 43.67254, 'longitude...",/v3/places/5859d15c1d21ba53f2f9b4ef,"{'address': '1471 Gerrard St E', 'country': 'C...",Lake Inez,{},America/Toronto,43.672346,-79.320804
2,4c1169396e5dc9b61b10b02d,"[{'id': 13031, 'name': 'Burger Joint', 'short_...",[],VeryLikelyOpen,377,"{'main': {'latitude': 43.666763, 'longitude': ...",/v3/places/4c1169396e5dc9b61b10b02d,"{'address': '1636 Queen St E', 'country': 'CA'...",The Burger's Priest,{},America/Toronto,43.666763,-79.315564
3,4ad9ebdcf964a520e61b21e3,"[{'id': 13049, 'name': 'Diner', 'short_name': ...",[],VeryLikelyOpen,129,"{'main': {'latitude': 43.665121, 'longitude': ...",/v3/places/4ad9ebdcf964a520e61b21e3,"{'address': '1483 Queen St E', 'country': 'CA'...",Chick-N-Joy,{},America/Toronto,43.665121,-79.321347
4,54302333498ed6ec0533ab40,"[{'id': 13140, 'name': 'Egyptian Restaurant', ...",[],VeryLikelyOpen,1001,"{'drop_off': {'latitude': 43.671737, 'longitud...",/v3/places/54302333498ed6ec0533ab40,"{'address': '226 Greenwood Ave', 'country': 'C...",Maha's Fine Egyptian Cuisine,{},America/Toronto,43.671682,-79.328462
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,55610dcd498e1b69839a65d9,"[{'id': 13002, 'name': 'Bakery', 'short_name':...",[],VeryLikelyOpen,682,"{'drop_off': {'latitude': 43.661132, 'longitud...",/v3/places/55610dcd498e1b69839a65d9,"{'address': '3 Westmoreland Ave', 'country': '...",Field Trip Cafe,{},America/Toronto,43.663880,-79.326107
6,56418c69498ec00425302f6a,"[{'id': 13263, 'name': 'Japanese Restaurant', ...",[],LikelyOpen,876,"{'main': {'latitude': 43.649644, 'longitude': ...",/v3/places/56418c69498ec00425302f6a,"{'address': '1330 Dundas St W', 'country': 'CA...",Imanishi Japanese Kitchen,{},America/Toronto,43.666623,-79.316861
7,5963b3e52be425495a1ed8a1,"[{'id': 13236, 'name': 'Italian Restaurant', '...",[],VeryLikelyOpen,777,"{'drop_off': {'latitude': 43.658444, 'longitud...",/v3/places/5963b3e52be425495a1ed8a1,"{'address': '1281 Bloor St W', 'country': 'CA'...",Sugo,{},America/Toronto,43.667418,-79.312824
8,4dc5bc33d164eb9c9ff7a2a3,"[{'id': 13002, 'name': 'Bakery', 'short_name':...",[],VeryLikelyOpen,828,"{'drop_off': {'latitude': 43.661695, 'longitud...",/v3/places/4dc5bc33d164eb9c9ff7a2a3,"{'address': '205 Delaware Ave', 'country': 'CA...",Bakerbots Baking,{},America/Toronto,43.668614,-79.317268


In [19]:
fsq_df1_normalized = json_normalize(fsq_df['categories'])
fsq_df1_normalized

,0,1,2
0,"{'id': 13199, 'name': 'Indian Restaurant', 'sh...",None,None
1,"{'id': 13003, 'name': 'Bar', 'short_name': 'Ba...","{'id': 13072, 'name': 'Asian Restaurant', 'sho...",None
2,"{'id': 13031, 'name': 'Burger Joint', 'short_n...","{'id': 13068, 'name': 'American Restaurant', '...","{'id': 13145, 'name': 'Fast Food Restaurant', ..."
3,"{'id': 13049, 'name': 'Diner', 'short_name': '...","{'id': 13145, 'name': 'Fast Food Restaurant', ...",None
4,"{'id': 13140, 'name': 'Egyptian Restaurant', '...",None,None
...,...,...,...
976,"{'id': 13002, 'name': 'Bakery', 'short_name': ...","{'id': 13006, 'name': 'Beer Bar', 'short_name'...","{'id': 13034, 'name': 'Café', 'short_name': 'C..."
977,"{'id': 13263, 'name': 'Japanese Restaurant', '...",None,None
978,"{'id': 13236, 'name': 'Italian Restaurant', 's...",None,None
979,"{'id': 13002, 'name': 'Bakery', 'short_name': ...","{'id': 13065, 'name': 'Restaurant', 'short_nam...",None


In [20]:
place = json_normalize(fsq_df1_normalized[0])
place

,id,name,short_name,plural_name,icon.prefix,icon.suffix
0,13199,Indian Restaurant,Indian,Indian Restaurants,https://ss3.4sqi.net/img/categories_v2/food/in...,.png
1,13003,Bar,Bar,Bars,https://ss3.4sqi.net/img/categories_v2/nightli...,.png
2,13031,Burger Joint,Burgers,Burger Joints,https://ss3.4sqi.net/img/categories_v2/food/bu...,.png
3,13049,Diner,Diner,Diners,https://ss3.4sqi.net/img/categories_v2/food/di...,.png
4,13140,Egyptian Restaurant,Egyptian Restaurant,Egyptian Restaurants,https://ss3.4sqi.net/img/categories_v2/food/mi...,.png
...,...,...,...,...,...,...
976,13002,Bakery,Bakery,Bakeries,https://ss3.4sqi.net/img/categories_v2/food/ba...,.png
977,13263,Japanese Restaurant,Japanese,Japanese Restaurants,https://ss3.4sqi.net/img/categories_v2/food/ja...,.png
978,13236,Italian Restaurant,Italian,Italian Restaurants,https://ss3.4sqi.net/img/categories_v2/food/it...,.png
979,13002,Bakery,Bakery,Bakeries,https://ss3.4sqi.net/img/categories_v2/food/ba...,.png


In [21]:
fsq_df['categories'] = place['name']
fsq_df

,fsq_id,categories,chains,closed_bucket,distance,geocodes,link,location,name,related_places,timezone,latitude,longitude
0,4ae0c7a8f964a520638221e3,Indian Restaurant,[],VeryLikelyOpen,783,"{'main': {'latitude': 43.67205, 'longitude': -...",/v3/places/4ae0c7a8f964a520638221e3,"{'address': '1460 Gerrard St E', 'country': 'C...",Udupi Palace,{'parent': {'fsq_id': '4d1bcc3234125941cd53cb1...,America/Toronto,43.672050,-79.322451
1,5859d15c1d21ba53f2f9b4ef,Bar,[],VeryLikelyOpen,809,"{'drop_off': {'latitude': 43.67254, 'longitude...",/v3/places/5859d15c1d21ba53f2f9b4ef,"{'address': '1471 Gerrard St E', 'country': 'C...",Lake Inez,{},America/Toronto,43.672346,-79.320804
2,4c1169396e5dc9b61b10b02d,Burger Joint,[],VeryLikelyOpen,377,"{'main': {'latitude': 43.666763, 'longitude': ...",/v3/places/4c1169396e5dc9b61b10b02d,"{'address': '1636 Queen St E', 'country': 'CA'...",The Burger's Priest,{},America/Toronto,43.666763,-79.315564
3,4ad9ebdcf964a520e61b21e3,Diner,[],VeryLikelyOpen,129,"{'main': {'latitude': 43.665121, 'longitude': ...",/v3/places/4ad9ebdcf964a520e61b21e3,"{'address': '1483 Queen St E', 'country': 'CA'...",Chick-N-Joy,{},America/Toronto,43.665121,-79.321347
4,54302333498ed6ec0533ab40,Egyptian Restaurant,[],VeryLikelyOpen,1001,"{'drop_off': {'latitude': 43.671737, 'longitud...",/v3/places/54302333498ed6ec0533ab40,"{'address': '226 Greenwood Ave', 'country': 'C...",Maha's Fine Egyptian Cuisine,{},America/Toronto,43.671682,-79.328462
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,55610dcd498e1b69839a65d9,Café,[],VeryLikelyOpen,682,"{'drop_off': {'latitude': 43.661132, 'longitud...",/v3/places/55610dcd498e1b69839a65d9,"{'address': '3 Westmoreland Ave', 'country': '...",Field Trip Cafe,{},America/Toronto,43.663880,-79.326107
6,56418c69498ec00425302f6a,Pizzeria,[],LikelyOpen,876,"{'main': {'latitude': 43.649644, 'longitude': ...",/v3/places/56418c69498ec00425302f6a,"{'address': '1330 Dundas St W', 'country': 'CA...",Imanishi Japanese Kitchen,{},America/Toronto,43.666623,-79.316861
7,5963b3e52be425495a1ed8a1,Ice Cream Parlor,[],VeryLikelyOpen,777,"{'drop_off': {'latitude': 43.658444, 'longitud...",/v3/places/5963b3e52be425495a1ed8a1,"{'address': '1281 Bloor St W', 'country': 'CA'...",Sugo,{},America/Toronto,43.667418,-79.312824
8,4dc5bc33d164eb9c9ff7a2a3,Fish and Chips Shop,[],VeryLikelyOpen,828,"{'drop_off': {'latitude': 43.661695, 'longitud...",/v3/places/4dc5bc33d164eb9c9ff7a2a3,"{'address': '205 Delaware Ave', 'country': 'CA...",Bakerbots Baking,{},America/Toronto,43.668614,-79.317268


In [22]:
fsq_df2_normalized = json_normalize(fsq_df['location'])

In [23]:
# Renaming formatted_address to location

fsq_df2_normalized.rename(columns = {'formatted_address': 'location'},inplace = True)
fsq_df2_normalized

,address,country,cross_street,location,locality,postcode,region,address_extended
0,1460 Gerrard St E,CA,Coxwell Ave,"1460 Gerrard St E (Coxwell Ave), Toronto ON M4...",Toronto,M4L 2A3,ON,NaN
1,1471 Gerrard St E,CA,,"1471 Gerrard St E, Toronto ON M4L 2A1",Toronto,M4L 2A1,ON,NaN
2,1636 Queen St E,CA,btwn Coxwell Ave. & Kingston Rd.,1636 Queen St E (btwn Coxwell Ave. & Kingston ...,Toronto,M4L 1G3,ON,NaN
3,1483 Queen St E,CA,,"1483 Queen St E, Toronto ON M4L 1E2",Toronto,M4L 1E2,ON,NaN
4,226 Greenwood Ave,CA,Sandford Avenue,"226 Greenwood Ave (Sandford Avenue), Toronto O...",Toronto,M4L 2R2,ON,NaN
...,...,...,...,...,...,...,...,...
976,3 Westmoreland Ave,CA,Bloor St W,"3 Westmoreland Ave (Bloor St W), Toronto ON M6...",Toronto,M6H 2Z6,ON,NaN
977,1330 Dundas St W,CA,at Rusholme Rd,"1330 Dundas St W (at Rusholme Rd), Toronto ON ...",Toronto,M6J 1Y1,ON,NaN
978,1281 Bloor St W,CA,Lansdowne,"1281 Bloor St W (Lansdowne), Toronto ON M6H 1N7",Toronto,M6H 1N7,ON,NaN
979,205 Delaware Ave,CA,at Bloor St. W,"205 Delaware Ave (at Bloor St. W), Toronto ON ...",Toronto,M6H 2T4,ON,NaN


In [24]:
fsq_df['location'] = fsq_df2_normalized['location']
fsq_df

,fsq_id,categories,chains,closed_bucket,distance,geocodes,link,location,name,related_places,timezone,latitude,longitude
0,4ae0c7a8f964a520638221e3,Indian Restaurant,[],VeryLikelyOpen,783,"{'main': {'latitude': 43.67205, 'longitude': -...",/v3/places/4ae0c7a8f964a520638221e3,"1460 Gerrard St E (Coxwell Ave), Toronto ON M4...",Udupi Palace,{'parent': {'fsq_id': '4d1bcc3234125941cd53cb1...,America/Toronto,43.672050,-79.322451
1,5859d15c1d21ba53f2f9b4ef,Bar,[],VeryLikelyOpen,809,"{'drop_off': {'latitude': 43.67254, 'longitude...",/v3/places/5859d15c1d21ba53f2f9b4ef,"1471 Gerrard St E, Toronto ON M4L 2A1",Lake Inez,{},America/Toronto,43.672346,-79.320804
2,4c1169396e5dc9b61b10b02d,Burger Joint,[],VeryLikelyOpen,377,"{'main': {'latitude': 43.666763, 'longitude': ...",/v3/places/4c1169396e5dc9b61b10b02d,1636 Queen St E (btwn Coxwell Ave. & Kingston ...,The Burger's Priest,{},America/Toronto,43.666763,-79.315564
3,4ad9ebdcf964a520e61b21e3,Diner,[],VeryLikelyOpen,129,"{'main': {'latitude': 43.665121, 'longitude': ...",/v3/places/4ad9ebdcf964a520e61b21e3,"1483 Queen St E, Toronto ON M4L 1E2",Chick-N-Joy,{},America/Toronto,43.665121,-79.321347
4,54302333498ed6ec0533ab40,Egyptian Restaurant,[],VeryLikelyOpen,1001,"{'drop_off': {'latitude': 43.671737, 'longitud...",/v3/places/54302333498ed6ec0533ab40,"226 Greenwood Ave (Sandford Avenue), Toronto O...",Maha's Fine Egyptian Cuisine,{},America/Toronto,43.671682,-79.328462
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,55610dcd498e1b69839a65d9,Café,[],VeryLikelyOpen,682,"{'drop_off': {'latitude': 43.661132, 'longitud...",/v3/places/55610dcd498e1b69839a65d9,"1347 Queen St E (Knox Aveneue), Toronto ON M4L...",Field Trip Cafe,{},America/Toronto,43.663880,-79.326107
6,56418c69498ec00425302f6a,Pizzeria,[],LikelyOpen,876,"{'main': {'latitude': 43.649644, 'longitude': ...",/v3/places/56418c69498ec00425302f6a,"6 Coxwell Ave (Queen), Toronto ON M4L 3A7",Imanishi Japanese Kitchen,{},America/Toronto,43.666623,-79.316861
7,5963b3e52be425495a1ed8a1,Ice Cream Parlor,[],VeryLikelyOpen,777,"{'drop_off': {'latitude': 43.658444, 'longitud...",/v3/places/5963b3e52be425495a1ed8a1,"1698 Queen St W, Scarborough ON M6R 1B3",Sugo,{},America/Toronto,43.667418,-79.312824
8,4dc5bc33d164eb9c9ff7a2a3,Fish and Chips Shop,[],VeryLikelyOpen,828,"{'drop_off': {'latitude': 43.661695, 'longitud...",/v3/places/4dc5bc33d164eb9c9ff7a2a3,"73 Coxwell Ave (Dundas St & Coxwell St), Toron...",Bakerbots Baking,{},America/Toronto,43.668614,-79.317268


In [25]:
# Dropping unwanted columns in the DataFrame

fsq_df.drop(columns = ['chains','geocodes','link','related_places','timezone'],axis = 1, inplace = True)
fsq_df

,fsq_id,categories,closed_bucket,distance,location,name,latitude,longitude
0,4ae0c7a8f964a520638221e3,Indian Restaurant,VeryLikelyOpen,783,"1460 Gerrard St E (Coxwell Ave), Toronto ON M4...",Udupi Palace,43.672050,-79.322451
1,5859d15c1d21ba53f2f9b4ef,Bar,VeryLikelyOpen,809,"1471 Gerrard St E, Toronto ON M4L 2A1",Lake Inez,43.672346,-79.320804
2,4c1169396e5dc9b61b10b02d,Burger Joint,VeryLikelyOpen,377,1636 Queen St E (btwn Coxwell Ave. & Kingston ...,The Burger's Priest,43.666763,-79.315564
3,4ad9ebdcf964a520e61b21e3,Diner,VeryLikelyOpen,129,"1483 Queen St E, Toronto ON M4L 1E2",Chick-N-Joy,43.665121,-79.321347
4,54302333498ed6ec0533ab40,Egyptian Restaurant,VeryLikelyOpen,1001,"226 Greenwood Ave (Sandford Avenue), Toronto O...",Maha's Fine Egyptian Cuisine,43.671682,-79.328462
...,...,...,...,...,...,...,...,...
5,55610dcd498e1b69839a65d9,Café,VeryLikelyOpen,682,"1347 Queen St E (Knox Aveneue), Toronto ON M4L...",Field Trip Cafe,43.663880,-79.326107
6,56418c69498ec00425302f6a,Pizzeria,LikelyOpen,876,"6 Coxwell Ave (Queen), Toronto ON M4L 3A7",Imanishi Japanese Kitchen,43.666623,-79.316861
7,5963b3e52be425495a1ed8a1,Ice Cream Parlor,VeryLikelyOpen,777,"1698 Queen St W, Scarborough ON M6R 1B3",Sugo,43.667418,-79.312824
8,4dc5bc33d164eb9c9ff7a2a3,Fish and Chips Shop,VeryLikelyOpen,828,"73 Coxwell Ave (Dundas St & Coxwell St), Toron...",Bakerbots Baking,43.668614,-79.317268


In [26]:
# Saving DataFrame to csv 

fsq_df.to_csv('fsq_df.csv', index = False)